# TAnoGAN

### Problem: Anomaly  Detection in Time Series Data

#### Technique: (a) Generative Adversarial Network (GAN) and (b) Anomaly Score by Mapping Real Data to Latent Space

Language: Python

Library: PyTorch

This is the code for TAnoGAN model by Bashar and Nayak et al. used for anomaly detection in time series data

Data at Kaggle: https://www.kaggle.com/boltzmannbrain/nab

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision
import torch.nn.init as init
from torch.autograd import Variable
import datetime
from nab_dataset import NabDataset
from models.recurrent_models import LSTMGenerator, LSTMDiscriminator

### Define Basic Settings

In [ ]:
class ArgsTrn:
    workers=4
    batch_size=32
    epochs=10
    lr=0.0002
    cuda = True
    outf='checkpoints'
    imf='images'
    manualSeed=2
opt_trn=ArgsTrn()

In [ ]:
torch.manual_seed(opt_trn.manualSeed)
cudnn.benchmark = True

### Setup the Training Data Loader

In [ ]:
end_name = 'exchange-4_cpm_results.csv'
data_file = 'data\\realAdExchange\\'+end_name
key = 'realAdExchange/'+end_name

In [ ]:
# settings for data loader
class DataSettings:
    def __init__(self):
        self.BASE = 'NabDataset\\'
        self.label_file = 'labels\\combined_windows.json'
        self.data_file = data_file
        self.key = key
        self.train = True
data_settings = DataSettings()

In [ ]:
dataset = NabDataset(data_settings=data_settings)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt_trn.batch_size,
                                         shuffle=False, num_workers=int(opt_trn.workers))

In [ ]:
dataset.x.shape, dataset.y.shape

### Setup Models

In [ ]:
device = torch.device("cuda:0" if opt_trn.cuda else "cpu")
seq_len = dataset.window_length
in_dim = nz = dataset.n_feature

In [ ]:
netD = LSTMDiscriminator(in_dim=in_dim, hidden_dim=100, device=device).to(device)
netG = LSTMGenerator(in_dim=in_dim, out_dim=in_dim, hidden_dim=100, device=device).to(device)

In [ ]:
print("|Discriminator Architecture|\n", netD)
print("|Generator Architecture|\n", netG)

In [ ]:
# Setup loss function
criterion = nn.BCELoss().to(device)

In [ ]:
# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt_trn.lr)
optimizerG = optim.Adam(netG.parameters(), lr=opt_trn.lr)

### Train models

In [ ]:
real_label = 1
fake_label = 0

for epoch in range(opt_trn.epochs):
    for i, (x,y) in enumerate(dataloader, 0):
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################

        #Train with real data
        netD.zero_grad()
        real = x.to(device)
        batch_size, seq_len = real.size(0), real.size(1)
        label = torch.full((batch_size, seq_len, 1), real_label, device=device)

        output = netD(real)
        errD_real = criterion(output, label)
        errD_real.backward()
        optimizerD.step()
        D_x = output.mean().item()
        
        #Train with fake data
        noise = Variable(init.normal(torch.Tensor(batch_size,seq_len,in_dim),mean=0,std=0.1)).cuda()
        fake = netG.forward(noise)
        output = netD.forward(fake.detach()) # detach causes gradient is no longer being computed or stored to save memeory
        label.fill_(fake_label)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        noise = Variable(init.normal(torch.Tensor(batch_size,seq_len,in_dim),mean=0,std=0.1)).cuda()
        fake = netG.forward(noise)
        label.fill_(real_label) 
        output = netD.forward(fake)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()
        D_G_z2 = output.mean().item()
        
    print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' 
          % (epoch, opt_trn.epochs, i, len(dataloader),
             errD.item(), errG.item(), D_x, D_G_z1, D_G_z2), end='')
    print()

## Anomaly Detection

In [ ]:
from torch.autograd import Variable
from nab_dataset import NabDataset
from torch.autograd import Variable
import torch
import torch.nn.init as init

##### In this step we can restore model for testing. For simplicity we are omiting that step

In [ ]:
generator = netG
discriminator = netD

##### Define Test Data

In [ ]:
# settings for data loader
class TestDataSettings:
    def __init__(self):
        self.BASE = 'NabDataset\\'
        self.label_file = 'labels\\combined_windows.json'
        self.data_file = data_file
        self.key = key
        self.train = False

test_data_settings = TestDataSettings()
test_dataset = NabDataset(test_data_settings)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=opt_test.batch_size, 
                                         shuffle=False, num_workers=int(opt_test.workers))

In [ ]:
test_dataset.x.shape, test_dataset.y.shape, test_dataset.data_len

##### Define a function to calculate anomaly score

In [ ]:
# Lambda = 0.1
# x is new data, G_z is closely regenerated data

def Anomaly_score(x, G_z, Lambda=0.1):
    residual_loss = torch.sum(torch.abs(x-G_z))
    
    x_feature = discriminator(x.to(device))#This can be updated as the paper
    G_z_feature = discriminator(G_z.to(device))#This can be updated as the paper
    
    discrimination_loss = torch.sum(torch.abs(x_feature-G_z_feature))
    
    total_loss = (1-Lambda)*residual_loss.to(device) + Lambda*discrimination_loss
    return total_loss

##### Test with the Data

In [ ]:
loss_list = []
y_list = []
for i, (x,y) in enumerate(test_dataloader):
    print(i, y)
    
    z = Variable(init.normal(torch.zeros(opt_test.batch_size,
                                     test_dataset.window_length, 
                                     test_dataset.n_feature),mean=0,std=0.1),requires_grad=True)
    #z = x
    z_optimizer = torch.optim.Adam([z],lr=1e-2)
    
    loss = None
    for j in range(50): # set your interation range
        gen_fake = generator(z.cuda())
        loss = Anomaly_score(Variable(x).cuda(), gen_fake)
        loss.backward()
        z_optimizer.step()

    loss_list.append(loss)
    y_list.append(y)
    print('~~~~~~~~loss={},  y={} ~~~~~~~~~~'.format(loss, y))
    #break

In [ ]:
import pandas as pd

THRESHOLD = 1

TIME_STEPS = dataset.window_length
test_score_df = pd.DataFrame(index=range(test_dataset.data_len))
test_score_df['loss'] = [loss.item()/test_dataset.window_length for loss in loss_list]
test_score_df['y'] = test_dataset.y
test_score_df['threshold'] = THRESHOLD
test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
test_score_df['t'] = [x[59].item() for x in test_dataset.x]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(test_score_df.index, test_score_df.loss, label='loss')
plt.plot(test_score_df.index, test_score_df.threshold, label='threshold')
plt.plot(test_score_df.index, test_score_df.y, label='y')
plt.xticks(rotation=25)
plt.legend();

In [ ]:
anomalies = test_score_df[test_score_df.anomaly == True]
anomalies.head()

In [ ]:
import seaborn as sns

plt.plot(
  range(test_dataset.data_len), 
  test_score_df['t'], 
  label='value'
);

sns.scatterplot(
  anomalies.index,
  anomalies.t,
  color=sns.color_palette()[3],
  s=52,
  label='anomaly'
)

plt.plot(
  range(len(test_score_df['y'])),
  test_score_df['y'],
  label='y'
)

plt.xticks(rotation=25)
plt.legend();

In [ ]:
import numpy as np
start_end = []
state = 0
for idx in test_score_df.index:
    if state==0 and test_score_df.loc[idx, 'y']==1:
        state=1
        start = idx
    if state==1 and test_score_df.loc[idx, 'y']==0:
        state = 0
        end = idx
        start_end.append((start, end))

for s_e in start_end:
    if sum(test_score_df[s_e[0]:s_e[1]+1]['anomaly'])>0:
        for i in range(s_e[0], s_e[1]+1):
            test_score_df.loc[i, 'anomaly'] = 1
            
actual = np.array(test_score_df['y'])
predicted = np.array([int(a) for a in test_score_df['anomaly']])

In [ ]:
import numpy as np
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

predicted = np.array(predicted)
actual = np.array(actual)

tp = np.count_nonzero(predicted * actual)
tn = np.count_nonzero((predicted - 1) * (actual - 1))
fp = np.count_nonzero(predicted * (actual - 1))
fn = np.count_nonzero((predicted - 1) * actual)

print('True Positive\t', tp)
print('True Negative\t', tn)
print('False Positive\t', fp)
print('False Negative\t', fn)

accuracy = (tp + tn) / (tp + fp + fn + tn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
fmeasure = (2 * precision * recall) / (precision + recall)
cohen_kappa_score = cohen_kappa_score(predicted, actual)
false_positive_rate, true_positive_rate, thresholds = roc_curve(actual, predicted)
auc_val = auc(false_positive_rate, true_positive_rate)
roc_auc_val = roc_auc_score(actual, predicted)

print('Accuracy\t', accuracy)
print('Precision\t', precision)
print('Recall\t', recall)
print('f-measure\t', fmeasure)
print('cohen_kappa_score\t', cohen_kappa_score)
print('auc\t', auc_val)
print('roc_auc\t', roc_auc_val)